In [73]:
REFERENCE_SIZES = {
    'BusinessDepartment': {'height': 15.0, 'width': 40.0, 'reference_pixel_height': 450},
    'Library': {'height': 11.0, 'width': 45.0, 'reference_pixel_height': 450},
    'EEDepartment': {'height': 12.0, 'width': 50.0, 'reference_pixel_height': 450},
    'CivilDepartment': {'height': 14.0, 'width': 50.0, 'reference_pixel_height': 500},
    'OldCSDepartment': {'height': 15.0, 'width': 48.0, 'reference_pixel_height': 450},
    'NewCSDepartment': {'height': 16.0, 'width': 50.0, 'reference_pixel_height': 500}
}

In [74]:
def load_recognition_model():
    """Loads the trained landmark recognition model"""
    import os
    import pandas as pd
    from tensorflow.keras.models import load_model
    import tensorflow as tf
    from sklearn.preprocessing import LabelEncoder

    model_path = 'model/landmark_recognition_model.h5'
    if not os.path.exists(model_path):
        model_path = 'landmark_recognition_model.h5'
        if not os.path.exists(model_path):
            raise FileNotFoundError("Model file not found. Please place 'landmark_recognition_model.h5' in the correct directory.")

    model = load_model(model_path)

    annotations_path = "annotations/annotations.csv"
    if not os.path.exists(annotations_path):
        annotations_path = "annotations.csv"
        if not os.path.exists(annotations_path):
            raise FileNotFoundError("Annotations CSV not found. Please place 'annotations.csv' in the correct directory.")

    df = pd.read_csv(annotations_path)
    label_encoder = LabelEncoder()
    label_encoder.fit(df['building_name'])

    base_model = tf.keras.applications.MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3),
        pooling='avg'
    )
    base_model.trainable = False

    return model, label_encoder, base_model

In [75]:
def load_and_preprocess_image(img_path):
    """Load and preprocess image for the model"""
    from tensorflow.keras.preprocessing import image
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
    import numpy as np

    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array, img

In [76]:
def recognize_landmark(img_path, model, label_encoder, base_model):
    """Recognize landmark in the given image"""
    import numpy as np

    preprocessed_img, original_img = load_and_preprocess_image(img_path)
    features = base_model.predict(preprocessed_img)
    prediction = model.predict(features)
    predicted_class = np.argmax(prediction[0])
    building_name = label_encoder.classes_[predicted_class]
    confidence = prediction[0][predicted_class]

    return building_name, confidence, original_img

In [77]:
def detect_building_boundaries(image_path):
    """Detect the boundaries of a building in the image"""
    import cv2
    import numpy as np

    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not read image at path: {image_path}")
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    dilated = cv2.dilate(edges, np.ones((3, 3), np.uint8), iterations=2)
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return None, 0, img_rgb

    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    cv2.rectangle(img_rgb, (x, y), (x+w, y+h), (0, 255, 0), 2)

    return (x, y, w, h), h, img_rgb

In [78]:
def estimate_distance_by_size(building_name, pixel_height):
    """Estimate the distance to a building based on its apparent size in pixels"""
    if building_name not in REFERENCE_SIZES:
        return None

    reference_data = REFERENCE_SIZES[building_name]
    reference_height_m = reference_data['height']
    reference_pixel_height = reference_data['reference_pixel_height']
    reference_distance = 10.0  # meters
    focal_length = (reference_pixel_height * reference_distance) / reference_height_m
    estimated_distance = (reference_height_m * focal_length) / pixel_height

    return estimated_distance

In [79]:
def estimate_location_by_trilateration(buildings, distances):
    """Estimate user location using trilateration from three buildings"""
    import numpy as np

    BUILDING_COORDINATES = {
        'BusinessDepartment': (250, 250),
        'EEDepartment': (350, 220),
        'OldCSDepartment': (250, 200),
        'Library': (250, 300),
        'CivilDepartment': (150, 300),
        'NewCSDepartment': (400, 250)
    }

    if len(buildings) != 3 or len(distances) != 3:
        return None

    # Check for duplicate buildings
    if len(set(buildings)) != 3:
        return None

    for building in buildings:
        if building not in BUILDING_COORDINATES:
            return None

    # Extract coordinates and distances
    x1, y1 = BUILDING_COORDINATES[buildings[0]]
    x2, y2 = BUILDING_COORDINATES[buildings[1]]
    x3, y3 = BUILDING_COORDINATES[buildings[2]]
    r1, r2, r3 = distances

    # Trilateration equations: solve for (x, y)
    A = np.array([
        [2*(x1 - x3), 2*(y1 - y3)],
        [2*(x2 - x3), 2*(y2 - y3)]
    ])
    B = np.array([
        [r3**2 - r1**2 - x3**2 + x1**2 - y3**2 + y1**2],
        [r3**2 - r2**2 - x3**2 + x2**2 - y3**2 + y2**2]
    ])

    try:
        X = np.linalg.solve(A, B)
        x, y = X[0][0], X[1][0]
        return (x, y)
    except np.linalg.LinAlgError:
        return None

In [80]:
def visualize_results(img_rgb, building_name, confidence, distance):
    """Display the image with detected building and estimated distance"""
    import matplotlib.pyplot as plt

    plt.figure(figsize)
    plt.imshow(img_rgb)
    plt.title(f"Detected: {building_name} (Confidence: {confidence:.2f})")
    plt.xlabel(f"Estimated Distance: {distance:.2f} meters")
    plt.axis('off')
    plt.show()

In [81]:
def estimate_location(building_names, distances, bearings=None):
    """Estimate user's location based on distances to landmarks"""
    import math
    import numpy as np

    BUILDING_COORDINATES = {
        'BusinessDepartment': (250, 250),
        'EEDepartment': (350, 220),
        'OldCSDepartment': (250, 200),
        'Library': (250, 300),
        'CivilDepartment': (150, 300),
        'NewCSDepartment': (400, 250)
    }

    if len(building_names) >= 3 and len(set(building_names)) >= 3:
        # Use trilateration for three distinct buildings
        location = estimate_location_by_trilateration(building_names[:3], distances[:3])
        if location is not None:
            return location

    # Fallback: Average single-building estimates
    locations = []
    for i, (building_name, distance) in enumerate(zip(building_names, distances)):
        if building_name not in BUILDING_COORDINATES or distance <= 0:
            continue
        building_x, building_y = BUILDING_COORDINATES[building_name]
        angle_rad = math.radians(bearings[i] if bearings and i < len(bearings) else 180)
        x = building_x - distance * math.sin(angle_rad)
        y = building_y + distance * math.cos(angle_rad)
        locations.append((x, y))

    if locations:
        return tuple(np.mean(locations, axis=0))
    return None

In [82]:
def visualize_location_on_map(user_location, building_coordinates, building_names=None, distances=None, filename=None):
    """Generate a static map showing user location and buildings"""
    import matplotlib.pyplot as plt
    import os
    import uuid
    import numpy as np

    fig, ax = plt.subplots(figsize=(10, 10))

    # Plot buildings
    for building_name, (x, y) in building_coordinates.items():
        ax.scatter(x, y, c='blue', marker='s', s=100, label=building_name if building_name not in ax.get_legend_handles_labels()[1] else "")
        ax.text(x, y + 10, building_name, fontsize=9, ha='center')

    # Plot user location if available
    if user_location:
        ax.scatter(user_location[0], user_location[1], c='red', marker='*', s=200, label='Estimated Location')

    # Plot distance circles if building_names and distances are provided
    if building_names and distances:
        for building_name, distance in zip(building_names, distances):
            if building_name in building_coordinates and distance > 0:
                center = building_coordinates[building_name]
                circle = plt.Circle(center, distance, fill=False, color='green', linestyle='--', alpha=0.5)
                ax.add_patch(circle)

    # Set plot limits with padding
    all_x = [x for x, y in building_coordinates.values()] + ([user_location[0]] if user_location else [])
    all_y = [y for x, y in building_coordinates.values()] + ([user_location[1]] if user_location else [])
    padding = 50
    ax.set_xlim(min(all_x) - padding, max(all_x) + padding)
    ax.set_ylim(min(all_y) - padding, max(all_y) + padding)

    ax.set_xlabel('X Coordinate (meters)')
    ax.set_ylabel('Y Coordinate (meters)')
    ax.set_title('Campus Map with Estimated Location')
    ax.legend()
    ax.grid(True)
    ax.set_aspect('equal', adjustable='box')

    if not filename:
        filename = f"map_{uuid.uuid4().hex}.png"
    save_path = os.path.join("static", "maps", filename)
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    plt.savefig(save_path, bbox_inches='tight', dpi=150)
    plt.close(fig)

    return filename

In [72]:
from flask import Flask, request, jsonify
import nest_asyncio
import os
import uuid

app = Flask(__name__)

@app.route('/process-images', methods=['POST'])
def process_images():
    try:
        model, label_encoder, base_model = load_recognition_model()
    except FileNotFoundError as e:
        return jsonify({'error': str(e)}), 500

    BUILDING_COORDINATES = {
        'BusinessDepartment': (250, 250),
        'EEDepartment': (350, 220),
        'OldCSDepartment': (250, 200),
        'Library': (250, 300),
        'CivilDepartment': (150, 300),
        'NewCSDepartment': (400, 250)
    }

    if 'images' not in request.files:
        return jsonify({'error': 'No images provided'}), 400

    images = request.files.getlist('images')
    if len(images) < 1 or len(images) > 3:
        return jsonify({'error': 'Please provide 1 to 3 images'}), 400

    results = []
    building_names = []
    distances = []

    for idx, img_file in enumerate(images):
        img_path = f'temp_image_{uuid.uuid4().hex}.jpg'
        img_file.save(img_path)

        try:
            building_name, confidence, original_img = recognize_landmark(img_path, model, label_encoder, base_model)
            bbox, pixel_height, img_rgb = detect_building_boundaries(img_path)
            distance = estimate_distance_by_size(building_name, pixel_height) if pixel_height > 0 else None

            result = {
                'image_index': idx,
                'building_name': building_name,
                'confidence': str(confidence),
                'bounding_box': str(bbox),
                'pixel_height': str(pixel_height),
                'distance': str(distance) if distance else 'None'
            }

            building_names.append(building_name)
            distances.append(distance if distance else 0)
            results.append(result)

        except Exception as e:
            results.append({
                'image_index': idx,
                'error': str(e)
            })
        finally:
            if os.path.exists(img_path):
                os.remove(img_path)

    user_location = estimate_location(building_names, distances)
    print(f"Estimated user location: {user_location}")

    # Generate static map even if location is None
    map_filename = visualize_location_on_map(
        user_location=user_location,
        building_coordinates=BUILDING_COORDINATES,
        building_names=building_names,
        distances=distances
    )
    map_url = f"http://192.168.1.18:5000/static/maps/{map_filename}"

    response = {
        'results': results,
        'user_location': str(user_location) if user_location else 'None',
        'map_url': map_url
    }
    print(f"Response: {response}")
    return jsonify(response)

if __name__ == '__main__':
    nest_asyncio.apply()
    app.run(host='0.0.0.0', port=5000, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.18:5000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.18:5000
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Estimated user location: (np.float64(250.0), np.float64(222.14944428916564))


192.168.1.17 - - [14/May/2025 00:29:43] "POST /process-images HTTP/1.1" 200 -
INFO:werkzeug:192.168.1.17 - - [14/May/2025 00:29:43] "POST /process-images HTTP/1.1" 200 -


Response: {'results': [{'image_index': 0, 'building_name': 'BusinessDepartment', 'confidence': '0.9298698', 'bounding_box': '(1851, 2470, 992, 346)', 'pixel_height': '346', 'distance': '13.00578034682081'}, {'image_index': 1, 'building_name': 'BusinessDepartment', 'confidence': '0.5673028', 'bounding_box': '(2780, 1948, 499, 628)', 'pixel_height': '628', 'distance': '7.165605095541402'}, {'image_index': 2, 'building_name': 'BusinessDepartment', 'confidence': '0.9228793', 'bounding_box': '(910, 2470, 1433, 71)', 'pixel_height': '71', 'distance': '63.38028169014085'}], 'user_location': '(np.float64(250.0), np.float64(222.14944428916564))', 'map_url': 'http://192.168.1.18:5000/static/maps/map_a1a932338d9f468dbe07654226bd474e.png'}


192.168.1.17 - - [14/May/2025 00:29:43] "GET /static/maps/map_a1a932338d9f468dbe07654226bd474e.png HTTP/1.1" 200 -
INFO:werkzeug:192.168.1.17 - - [14/May/2025 00:29:43] "GET /static/maps/map_a1a932338d9f468dbe07654226bd474e.png HTTP/1.1" 200 -
192.168.1.17 - - [14/May/2025 00:29:43] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:192.168.1.17 - - [14/May/2025 00:29:43] "GET /favicon.ico HTTP/1.1" 404 -
192.168.1.18 - - [14/May/2025 00:30:06] "GET /static/maps/map_a1a932338d9f468dbe07654226bd474e.png HTTP/1.1" 200 -
INFO:werkzeug:192.168.1.18 - - [14/May/2025 00:30:06] "GET /static/maps/map_a1a932338d9f468dbe07654226bd474e.png HTTP/1.1" 200 -
192.168.1.18 - - [14/May/2025 00:30:06] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:192.168.1.18 - - [14/May/2025 00:30:06] "GET /favicon.ico HTTP/1.1" 404 -
